# Arquitetura de Agente Autônomo para Navegação em Grafo (Livro-Jogo)

Este notebook implementa um agente autônomo projetado para navegar em uma narrativa de RPG estruturada como um livro-jogo. A estrutura do livro-jogo funciona como um **grafo direcionado**, onde cada página é um **nó** e as escolhas são as **arestas** que conectam os nós.

O objetivo principal é testar e validar a capacidade do agente de:
1.  **Observar** o estado atual (o texto da página).
2.  **Orientar-se** com base em suas instruções e na sua "ficha de personagem" (seu estado interno).
3.  **Decidir** qual a melhor ação (escolha) a tomar.
4.  **Agir** para transitar para o próximo estado (a próxima página).

Este ciclo é a implementação prática do modelo arquitetural **OODA (Observe, Orient, Decide, Act)**, um framework conceitual para tomada de decisão em cenários dinâmicos. Cada bloco de código abaixo representa um componente fundamental desta arquitetura.

### 1. Definição da Ficha de Personagem e Setup

O primeiro passo é modelar o estado do agente. A `create_character_sheet` funciona como um template para a ficha de um novo personagem, definindo todos os atributos possíveis, como informações básicas, perícias, recursos e inventário.

A função `setup_character` personaliza essa ficha com base na ocupação escolhida, ajustando as perícias e definindo os recursos iniciais, como a Sorte (`luck`). Isso garante que o agente comece o jogo com o estado inicial correto para a sua classe.

In [ ]:
import random

def create_character_sheet():
    """Cria um template para a ficha de personagem."""
    return {
        "info": {
            "name": "Character Name",
            "occupation": None,
            "age": 30,
            "backstory": ""
        },
        "contacts": {},
        "case_files": [],
        "magic": {"spells": [], "signare": []},
        "characteristics": {
            "STR": {"full": 0, "half": 0}, "CON": {"full": 0, "half": 0},
            "DEX": {"full": 0, "half": 0}, "INT": {"full": 0, "half": 0},
            "POW": {"full": 0, "half": 0}
        },
        "resources": {
            "luck": {"starting": 0, "current": 0},
            "magic_pts": {"starting": 0, "current": 0},
            "mov": 8
        },
        "skills": {
            "common": {
                "Athletics": {"full": 30, "half": 15}, "Drive": {"full": 30, "half": 15},
                "Navigate": {"full": 30, "half": 15}, "Observation": {"full": 30, "half": 15},
                "Read Person": {"full": 30, "half": 15}, "Research": {"full": 30, "half": 15},
                "Social": {"full": 30, "half": 15}, "Stealth": {"full": 30, "half": 15},
            },
            "combat": {
                "Fighting": {"full": 30, "half": 15}, "Firearms": {"full": 30, "half": 15}
            },
            "expert": {}
        },
        "status": {
            "damage_levels": ["Hurt", "Bloodied", "Down", "Impaired"],
            "current_damage": None,
            "damage_taken": 0
        },
        "inventory": {"equipment": [], "weapons": []},
        "page_history": []
    }

def setup_character(sheet, name, occupation, backstory):
    """Configura a ficha de personagem com base na ocupação e história."""
    sheet["info"]["name"] = name
    sheet["info"]["occupation"] = occupation
    sheet["info"]["backstory"] = backstory

    luck_roll = random.randint(1, 10) + random.randint(1, 10) + 50
    sheet["resources"]["luck"]["starting"] = luck_roll
    sheet["resources"]["luck"]["current"] = luck_roll

    if occupation == "Police Officer":
        for skill in ["Law", "Social", "Athletics", "Fighting"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Law"] = {"full": 60, "half": 30}
    elif occupation == "Social Worker":
        for skill in ["Observation", "Research", "Social"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}
    elif occupation == "Nurse":
        for skill in ["Observation", "Read Person", "Social"]:
            if skill in sheet["skills"]["common"]:
                sheet["skills"]["common"][skill] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Medicine"] = {"full": 60, "half": 30}
        sheet["skills"]["expert"]["Magic"] = {"full": 60, "half": 30}

    return sheet

### 2. Mecânica de Rolagem de Dados (D100)

O núcleo do sistema de RPG é a rolagem de dados. A função `make_check` simula uma rolagem de dado de 100 lados (D100) e a compara com um valor-alvo (a perícia do personagem).

Ela implementa a lógica de sucesso em diferentes níveis:
- **Critical Success**: Um resultado de 1.
- **Hard Success**: Um resultado menor ou igual à metade do valor da perícia.
- **Success**: Um resultado menor ou igual ao valor total da perícia.
- **Failure**: Um resultado maior que o valor da perícia.
- **Fumble**: Um resultado de 100.

Essa função é crucial para determinar o resultado de ações que envolvem risco ou incerteza.

In [ ]:
def make_check(target_value, half_value, bonus_die=False, penalty_die=False):
    """Realiza um teste de perícia D100 e retorna o nível de sucesso."""
    tens_roll_1 = random.randint(0, 9) * 10
    tens_roll_2 = random.randint(0, 9) * 10
    units_roll = random.randint(1, 10)

    if bonus_die:
        final_tens = min(tens_roll_1, tens_roll_2)
    elif penalty_die:
        final_tens = max(tens_roll_1, tens_roll_2)
    else:
        final_tens = tens_roll_1

    if final_tens == 0 and units_roll == 10:
        final_roll = 100
    elif final_tens == 0:
        final_roll = units_roll
    else:
        final_roll = final_tens + (units_roll % 10)

    if final_roll == 1:
        return ("Critical Success", final_roll)
    if final_roll == 100:
        return ("Fumble", final_roll)
    if final_roll <= half_value:
        return ("Hard Success", final_roll)
    if final_roll <= target_value:
        return ("Success", final_roll)
    
    return ("Failure", final_roll)

### 3. Encapsulamento dos Dados e Instruções do Jogo

Para uma arquitetura limpa e modular, os dados do jogo (o grafo de páginas) e as instruções para o agente são encapsulados em suas próprias classes: `GameData` e `GameInstructions`.

-   **GameInstructions**: Fornece o "backstory" ou as diretrizes de alto nível que o agente (por meio de um LLM) seguirá. Isso separa as regras do jogo da lógica do agente.
-   **GameData**: Contém o dicionário `PAGES`, que é a representação completa do livro-jogo. Cada chave é um ID de página, e o valor contém o texto e as escolhas possíveis. Este dicionário é carregado a partir do arquivo [`pages.py`](./pages.py).

Essa abordagem, conhecida como **Injeção de Dependência**, permite que o `Agent` seja independente dos dados específicos do jogo, tornando o sistema mais flexível e testável.

In [ ]:
class GameInstructions:
    def get_backstory(self):
        return """
        You are an AI agent playing a role-playing game.
        Your goal is to navigate the story, make decisions, and interact with the world.
        You must follow the rules of the game, manage your character's resources, and make choices that align with your character's personality and goals.
        The game is presented as a series of pages, each with a description of the current situation and a list of choices.
        Your task is to choose the best action to take in each situation.
        """

class GameData:
    def __init__(self):
        import pages
        self.PAGES = pages.PAGES

    def get_page(self, page_id):
        return self.PAGES.get(page_id)

### 4. A Classe `Agent`: O Coração do Sistema

A classe `Agent` é onde a lógica de decisão e ação reside. Ela é o cérebro do nosso jogador autônomo e implementa o ciclo OODA.

-   **`__init__`**: Inicializa o agente, criando sua ficha de personagem e recebendo os dados e instruções do jogo (via injeção de dependência).
-   **`_llm_decide`**: Esta é a fase de **Decisão**. Atualmente, ela usa uma lógica simples (escolhe a primeira opção válida), mas é aqui que um modelo de linguagem (LLM) seria integrado para tomar decisões mais inteligentes com base no texto da página e no estado do personagem.
-   **`perform_action`**: Esta é a fase de **Ação**. Ela processa a escolha feita, executa rolagens de dados, resolve combates e determina qual será a próxima página (o próximo estado).
-   **`run`**: O motor do jogo. Ele executa o loop principal: obtém a página atual (**Observação**), chama `_llm_decide` para escolher uma ação (**Orientação/Decisão**) e usa `perform_action` para executar a escolha e avançar na história (**Ação**).
-   **Métodos de Recurso**: Funções como `spend_luck` e `take_damage` gerenciam o estado interno do agente, modificando sua ficha de personagem conforme o jogo progride.

In [ ]:
class Agent:
    def __init__(self, name, occupation, game_instructions, game_data):
        base_sheet = create_character_sheet()
        self.sheet = setup_character(base_sheet, name, occupation, game_instructions.get_backstory())
        self.game_data = game_data
        self.current_page = 1

    def __repr__(self):
        return f"Agent(Name: {self.sheet['info']['name']}, Occupation: {self.sheet['info']['occupation']})"

    def _llm_decide(self, page_text, choices):
        """Placeholder for LLM decision making. For now, chooses the first valid option."""
        print("--- AGENT DECISION ---")
        print(f"Based on the situation: '{page_text[:100]}...'")
        
        # New logic: Check for damage text before making a choice
        if "sofre * dano" in page_text:
            print("Agent recognizes that damage was taken from page text.")
            self.apply_damage(1)

        valid_choices = []
        for choice in choices:
            choice_text = choice.get("text", "").lower()
            occupation = self.sheet["info"]["occupation"].lower()
            
            # Conditional choices based on occupation
            if "se você é um" in choice_text:
                if occupation in choice_text:
                    valid_choices.append(choice)
            # Conditional choices based on damage
            elif "se o total de dano for" in choice_text:
                damage_taken = self.sheet["status"]["damage_taken"]
                try:
                    # Handles "Se o total de dano for 2"
                    if f"for {damage_taken}" in choice_text:
                        valid_choices.append(choice)
                    # Handles "Se o total de dano for 3 ou mais"
                    elif "ou mais" in choice_text:
                        num = int(choice_text.split(" for ")[1].split(" ")[0])
                        if damage_taken >= num:
                            valid_choices.append(choice)
                    # Handles "Se o dano total for 1 ou 2"
                    elif "1 ou 2" in choice_text and damage_taken in [1, 2]:
                        valid_choices.append(choice)
                except (ValueError, IndexError):
                    pass # Ignore if parsing fails
            else:
                valid_choices.append(choice)
        
        if not valid_choices:
            # Fallback to choices not restricted by occupation or damage
            valid_choices = [c for c in choices if "se você é um" not in c.get("text", "").lower() and "se o total de dano for" not in c.get("text", "").lower()]

        if not valid_choices:
            print("No valid choices found for the agent.")
            return None

        print("Available choices:")
        for choice in valid_choices:
            log_message = choice.get('text', '')
            if log_message and 'goto' in choice:
                log_message += f" -> To Page {choice['goto']}"
            elif not log_message:
                if 'goto' in choice:
                    log_message = f"Go to page {choice['goto']}"
                elif 'roll' in choice:
                    log_message = f"Attempt a '{choice['roll']}' skill check"
                elif 'luck_roll' in choice:
                    log_message = "Consider using Luck"
                elif 'opposed_roll' in choice:
                    log_message = f"Engage in opposed roll: {choice['opposed_roll']}"
                else:
                    log_message = "Follow the next step"
            print(f"- {log_message}")

        chosen = valid_choices[0].copy()

        if "luck_roll" in chosen and chosen["luck_roll"]:
            if self.sheet["resources"]["luck"]["current"] > 0:
                print("Agent decides to spend luck.")
                chosen["use_luck"] = True
            else:
                print("Agent has no luck to spend.")
                chosen["use_luck"] = False
        
        if "opposed_roll" in chosen:
            tactics = [k.split('_')[1] for k in chosen if k.startswith('win_')]
            if tactics:
                chosen_tactic = tactics[0]
                chosen["tactic"] = chosen_tactic
                print(f"Agent decides to use tactic: {chosen_tactic}")

        log_message = chosen.get('text', '')
        if log_message and 'goto' in chosen:
            log_message += f" -> To Page {chosen['goto']}"
        elif not log_message:
            if 'goto' in chosen:
                log_message = f"Go to page {chosen['goto']}"
            elif 'roll' in chosen:
                log_message = f"Attempt a '{chosen['roll']}' skill check"
            elif 'luck_roll' in chosen:
                log_message = "Consider using Luck"
            elif 'opposed_roll' in chosen:
                log_message = f"Engage in opposed roll: {chosen['opposed_roll']}"
            else:
                log_message = "Follow the next step"

        print(f"Agent chose: '{log_message}'")
        return chosen

    def perform_action(self, choice):
        """Processes the chosen action and returns the next page ID."""
        if not choice:
            return None

        if "goto" in choice and "roll" not in choice:
            return choice["goto"]
        
        if "roll" in choice:
            skill_to_roll = choice["roll"].split(" ")[0]
            
            skill_values = None
            if skill_to_roll in self.sheet["skills"]["common"]:
                skill_values = self.sheet["skills"]["common"][skill_to_roll]
            elif skill_to_roll in self.sheet["skills"]["combat"]:
                skill_values = self.sheet["skills"]["combat"][skill_to_roll]
            elif skill_to_roll in self.sheet["skills"]["expert"]:
                skill_values = self.sheet["skills"]["expert"][skill_to_roll]
            else:
                skill_values = self.sheet["characteristics"].get(skill_to_roll, {"full": 30, "half": 15})

            result, roll_value = make_check(skill_values["full"], skill_values["half"])
            print(f"Agent rolled for {skill_to_roll}: {roll_value} -> {result}")

            if result in ("Failure", "Fumble") and "failure_effect" in choice:
                if "sofre 1 de dano" in choice["failure_effect"]:
                    print("Agent recognizes that damage was taken from a failed roll.")
                    self.apply_damage(1)

            if result == "Critical Success":
                return choice.get("critical_success") or choice.get("hard_success") or choice.get("success")
            elif result == "Hard Success":
                return choice.get("hard_success") or choice.get("success")
            elif result == "Success":
                return choice.get("success")
            else: # Failure or Fumble
                return choice.get("fumble") or choice.get("failure")

        if "luck_roll" in choice:
            if choice.get("use_luck"):
                self.spend_luck(20)
                return choice["success"]
            else:
                return choice["failure"]

        if "opposed_roll" in choice:
            agent_skill_name = choice["opposed_roll"].split(" ")[0]
            agent_skill_values = self.sheet["skills"]["combat"].get(agent_skill_name, {"full": 30, "half": 15})
            agent_result, agent_roll = make_check(agent_skill_values["full"], agent_skill_values["half"])
            print(f"Agent's Roll ({agent_skill_name}): {agent_roll} -> {agent_result}")

            opponent_skill_values = {"full": 40, "half": 20} 
            opponent_result, opponent_roll = make_check(opponent_skill_values["full"], opponent_skill_values["half"])
            print(f"Opponent's Roll: {opponent_roll} -> {opponent_result}")

            success_ranking = {"Critical Success": 4, "Hard Success": 3, "Success": 2, "Failure": 1, "Fumble": 0}
            agent_rank = success_ranking.get(agent_result, 0)
            opponent_rank = success_ranking.get(opponent_result, 0)

            next_page = None
            if agent_rank > opponent_rank:
                tactic = choice.get("tactic")
                if tactic and f"win_{tactic}" in choice:
                    next_page = choice[f"win_{tactic}"]
                else:
                    next_page = choice["win"]
            elif agent_rank < opponent_rank:
                next_page = choice["lose"]
            else:
                next_page = choice.get("draw", choice["lose"])
            
            print(f"Action result: Moving to page {next_page}")
            return next_page

        print("Action result: No valid next page determined.")
        return None

    def run(self):
        """Main game loop for the agent."""
        while self.current_page is not None:
            if self.sheet["status"]["current_damage"] == "Impaired":
                print("\n--- CHARACTER IS IMPAIRED ---")
                print("The character has taken 4 or more damage and can no longer continue.")
                self.current_page = 999

            self.sheet["page_history"].append(self.current_page)
            page = self.game_data.get_page(self.current_page)
            if not page:
                print(f"Page {self.current_page} not found. Ending game.")
                break

            print(f"\n--- PAGE {self.current_page} ---")
            print(page["text"])

            if not page["choices"]:
                print("\n--- END OF STORY ---")
                break

            choice = self._llm_decide(page["text"], page["choices"])
            self.current_page = self.perform_action(choice)
            
            if self.current_page is None:
                print("Agent reached a dead end.")

    def spend_luck(self, amount):
        if self.sheet["resources"]["luck"]["current"] >= amount:
            self.sheet["resources"]["luck"]["current"] -= amount
            print(f"Spent {amount} luck. Current luck: {self.sheet['resources']['luck']['current']}")
            return True
        print("Not enough luck to spend.")
        return False

    def spend_magic(self, cost):
        if self.sheet["resources"]["magic_pts"]["current"] >= cost:
            self.sheet["resources"]["magic_pts"]["current"] -= cost
            print(f"Spent {cost} magic points. Current magic points: {self.sheet['resources']['magic_pts']['current']}")
            return True
        print("Not enough magic points.")
        return False

    def apply_damage(self, amount):
        """Applies damage to the character and updates their status."""
        self.sheet["status"]["damage_taken"] += amount
        total_damage = self.sheet["status"]["damage_taken"]
        
        new_status = None
        if total_damage >= 4:
            new_status = "Impaired"
        elif total_damage == 3:
            new_status = "Down"
        elif total_damage == 2:
            new_status = "Bloodied"
        elif total_damage == 1:
            new_status = "Hurt"
        
        if new_status and self.sheet["status"]["current_damage"] != new_status:
            self.sheet["status"]["current_damage"] = new_status
            print(f"Character took {amount} damage. Total damage: {total_damage}. Status is now {new_status}.")
        else:
            print(f"Character took {amount} damage. Total damage: {total_damage}.")

### 5. Ponto de Entrada: Executando o Jogo

Este é o ponto de entrada (`main`) do script. É aqui que tudo se junta:

1.  **Instanciação**: Os objetos `GameData` e `GameInstructions` são criados.
2.  **Criação do Agente**: Um `Agent` é criado com um nome, uma ocupação e as dependências recém-criadas. Você pode facilmente alterar a `player_occupation` para testar diferentes caminhos na história.
3.  **Execução**: O método `agent.run()` é chamado, iniciando o loop principal do jogo e permitindo que o agente comece sua jornada pela narrativa.

In [ ]:
if __name__ == '__main__':
    # 1. Instanciar os dados e as instruções do jogo
    game_data = GameData()
    game_instructions = GameInstructions()

    # 2. Criar o agente, injetando as dependências
    player_name = "Alex"
    player_occupation = "Police Officer" # Mude para "Social Worker" ou "Nurse" para testar outros caminhos
    agent = Agent(player_name, player_occupation, game_instructions, game_data)

    # 3. Rodar o loop principal do jogo
    agent.run()